In [1]:
import re
import json

# Simulación de base de datos IATA (puedes ampliarla)
IATA_CODES = {
    "quito": "UIO",
    "madrid": "MAD",
    "berlin": "BER",
    "frankfurt": "FRA",
    "barcelona": "BCN",
    "roma": "FCO",
    "sevilla": "SVQ"
}

# Lista de aerolíneas básicas (amplía según necesites)
AEROLINEAS = ["iberia", "lufthansa", "aireuropa", "latam", "avianca"]

# Mapeo de números escritos a enteros
CANTIDADES_PALABRA = {
    "un": 1, "uno": 1, "dos": 2, "tres": 3, "cuatro": 4,
    "cinco": 5, "seis": 6, "siete": 7, "ocho": 8, "nueve": 9, "diez": 10
}

MESES = [
    "enero","febrero","marzo","abril","mayo","junio",
    "julio","agosto","septiembre","octubre","noviembre","diciembre"
]

def extraer_datos(frase):
    frase_low = frase.lower()
    datos = {'origen': None, 'destino': None, 'fecha': None, 'cantidad': 1, 'aerolínea': None}

    # 1) Aerolínea: buscar por lista
    for aero in AEROLINEAS:
        if re.search(r'\b' + re.escape(aero) + r'\b', frase_low):
            datos['aerolínea'] = aero.capitalize()
            break

    # 2) Cantidad: buscar dígitos o palabras
    num_match = re.search(r'\b(\d+)\b', frase_low)
    if num_match:
        datos['cantidad'] = int(num_match.group(1))
    else:
        for palabra, val in CANTIDADES_PALABRA.items():
            if re.search(r'\b' + re.escape(palabra) + r'\b', frase_low):
                datos['cantidad'] = val
                break

    # 3) Origen y destino: intentar patrón "de X a Y" (captura nombres con espacios)
    patrón_de_a = re.search(r'de\s+([a-záéíóúñ ]+?)\s+a\s+([a-záéíóúñ ]+?)(?:\s+con|\s+para|\s*$)', frase_low, re.IGNORECASE)
    if patrón_de_a:
        datos['origen'] = patrón_de_a.group(1).strip().title()
        datos['destino'] = patrón_de_a.group(2).strip().title()
    else:
        # Si no coincide, buscar origen tras 'de' y destino tras 'a' por separado
        origen = re.search(r'de\s+([a-záéíóúñ ]+?)(?:\s+con|\s+para|\s+a\s+|$)', frase_low, re.IGNORECASE)
        destino = re.search(r'a\s+([a-záéíóúñ ]+?)(?:\s+con|\s+para|\s+el|\s+\d+|$)', frase_low, re.IGNORECASE)
        if origen:
            datos['origen'] = origen.group(1).strip().title()
        if destino:
            datos['destino'] = destino.group(1).strip().title()

    # 4) Fecha: buscar patrones "15 de octubre", "el 5 de agosto", o solo mes "en agosto"
    fecha_pattern = re.search(r'\b(\d{1,2}\s+de\s+[a-záéíóúñ]+)\b', frase_low)
    if fecha_pattern:
        datos['fecha'] = fecha_pattern.group(1)
    else:
        mes_pattern = re.search(r'\b(en\s+)?(' + '|'.join(MESES) + r')\b', frase_low)
        if mes_pattern:
            datos['fecha'] = mes_pattern.group(2).capitalize()

    return datos

def generar_json_final(datos):
    origen = datos.get('origen')
    destino = datos.get('destino')
    iata_from = IATA_CODES.get(origen.lower() if origen else "", "N/A")
    iata_to = IATA_CODES.get(destino.lower() if destino else "", "N/A")

    json_final = {
        "Origen": origen,
        "Ciudad Destino": destino,
        "IATA From": iata_from,
        "IATA To": iata_to,
        "Fecha": datos.get('fecha'),
        "Pax": datos.get('cantidad')
    }
    return json_final

def asistent():
    nombre_empresa = "TravelBot Salesiana"
    print(f"Hola, bienvenido a '{nombre_empresa}'. ¿Cómo te puedo ayudar?")
    while True:
        usuario_input = input(">> ").strip()
        if usuario_input.lower() == "salir":
            print("Gracias por usar nuestro servicio. ¡Hasta pronto!")
            break
        resultado = extraer_datos(usuario_input)
        print("\nPerfecto, comienzo la búsqueda de tu viaje a "
              f"{resultado['destino']} desde {resultado['origen']} para el {resultado['fecha']} "
              f"con {resultado['aerolínea']}.\n")
        json_final = generar_json_final(resultado)
        print("JSON Generado para el sistema de reservas:")
        print(json.dumps(json_final, indent=4, ensure_ascii=False))
        print("-" * 30)

# Ejemplos de prueba (puedes ejecutar estas líneas en otra celda)
ejemplos = [
    "Comprar tres billetes para el 15 de octubre con Iberia de Quito a Madrid",
    "Billete de Berlin a Madrid con Lufthansa",
    "Quiero 2 billetes de Madrid a Frankfurt en Septiembre",
    "Comprar billete Barcelona a Roma para el 25 de Agosto con Iberia",
    "Billete barato AirEuropa de Madrid a Sevilla"
]
for e in ejemplos:
    print("Frase:", e)
    print("Salida:", extraer_datos(e))
    print()

Frase: Comprar tres billetes para el 15 de octubre con Iberia de Quito a Madrid
Salida: {'origen': 'Octubre Con Iberia De Quito', 'destino': 'Madrid', 'fecha': '15 de octubre', 'cantidad': 15, 'aerolínea': 'Iberia'}

Frase: Billete de Berlin a Madrid con Lufthansa
Salida: {'origen': 'Berlin', 'destino': 'Madrid', 'fecha': None, 'cantidad': 1, 'aerolínea': 'Lufthansa'}

Frase: Quiero 2 billetes de Madrid a Frankfurt en Septiembre
Salida: {'origen': 'Madrid', 'destino': 'Frankfurt En Septiembre', 'fecha': 'Septiembre', 'cantidad': 2, 'aerolínea': None}

Frase: Comprar billete Barcelona a Roma para el 25 de Agosto con Iberia
Salida: {'origen': 'Agosto', 'destino': 'A Roma', 'fecha': '25 de agosto', 'cantidad': 25, 'aerolínea': 'Iberia'}

Frase: Billete barato AirEuropa de Madrid a Sevilla
Salida: {'origen': 'Madrid', 'destino': 'Sevilla', 'fecha': None, 'cantidad': 1, 'aerolínea': 'Aireuropa'}



In [ ]:
asistent()

Hola, bienvenido a 'TravelBot Salesiana'. ¿Cómo te puedo ayudar?
>> hola que tal

Perfecto, comienzo la búsqueda de tu viaje a Que Tal desde None para el None con None.

JSON Generado para el sistema de reservas:
{
    "Origen": null,
    "Ciudad Destino": "Que Tal",
    "IATA From": "N/A",
    "IATA To": "N/A",
    "Fecha": null,
    "Pax": 1
}
------------------------------
>> madrid

Perfecto, comienzo la búsqueda de tu viaje a None desde None para el None con None.

JSON Generado para el sistema de reservas:
{
    "Origen": null,
    "Ciudad Destino": null,
    "IATA From": "N/A",
    "IATA To": "N/A",
    "Fecha": null,
    "Pax": 1
}
------------------------------
>> Madrid

Perfecto, comienzo la búsqueda de tu viaje a None desde None para el None con None.

JSON Generado para el sistema de reservas:
{
    "Origen": null,
    "Ciudad Destino": null,
    "IATA From": "N/A",
    "IATA To": "N/A",
    "Fecha": null,
    "Pax": 1
}
------------------------------
>> Compra cuatro bill